In [ ]:
import numpy as np

#single_machine_heatmap : ./single_machine_heatmap.sh
with open("single_machine_heatmap", "r") as f:
    lines = f.readlines()
    f.close()

In [ ]:
# In order : name load_time method blur sobel (blur+sobel) export
print(lines[25])
print(lines[26])

In [ ]:
line = lines[0]
terms = line.split(" ")
file = terms[0].split("/")[-1]
file = file.split(".")[0]
width, height = file.split("x")
load_time = float(terms[1])
method = terms[2]
blur_time = float(terms[3])
sobel_time = float(terms[4])
filter_time = float(terms[5])
export_time = float(terms[6])

In [ ]:
print(width, height, load_time, method, blur_time, sobel_time, filter_time, export_time)

In [ ]:
total_test_per_file = 2
n = len(lines) // total_test_per_file

height_min = 1000
step = 100

openmp = np.zeros((9, 9, 5))
cuda = np.zeros((9, 9, 5))

for line in lines:
    terms = line.split(" ")
    file = terms[0].split("/")[-1]
    file = file.split(".")[0]
    
    width, height = map(int, file.split("x"))
    load_time = float(terms[1])
    method = terms[2]
    blur_time = float(terms[3])
    sobel_time = float(terms[4])
    filter_time = float(terms[5])
    export_time = float(terms[6])
    times = np.array([load_time, blur_time, sobel_time, filter_time, export_time])
    
    if method == "CUDA":
        i = (width - 1000) // 500
        j = (height - 1000) // 500
        cuda[j][i] = times
    elif "OpenMP" in method:
        i = (width - 1000) // 500
        j = (height - 1000) // 500
        openmp[j][i] = times

In [ ]:
import matplotlib.pyplot as plt

## Different time

In [ ]:
name = ["loading", "blur", "sobel", "filters", "export"]
for i in range(5):
    plt.figure()
    im = plt.imshow(cuda[:, :, i] - openmp[:, :, i], cmap="rainbow")
    plt.colorbar(im)
    plt.title("Difference {} time\n greater : better using openmp".format(name[i]))
    plt.xticks([i for i in range(0, 9, 2)], labels=[1000 + 500 * i for i in range(0, 9, 2)])
    plt.yticks([i for i in range(0, 9, 2)], labels=[1000 + 500 * i for i in range(0, 9, 2)])
    plt.show()